<a href="https://colab.research.google.com/github/z-arabi/Machine-Learning/blob/main/Ensemble_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/z-arabi/Machine-Learning.git
%cd "Machine-Learning"

Cloning into 'Machine-Learning'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 82 (delta 33), reused 10 (delta 1), pack-reused 0
Unpacking objects: 100% (82/82), done.
/content/Machine-Learning


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
%matplotlib inline

In [3]:
df = pd.read_csv('Datasets/Social_Network_Ads.csv')

df.drop('User ID', axis=1, inplace=True)
df.head(5)

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [4]:
df.Gender = pd.get_dummies(df.Gender, drop_first=True)
X = df.to_numpy()

np.random.seed = 0
X = X[np.random.permutation(X.shape[0])]

y = X[:, -1]
X = X[:, :-1]

In [5]:
split = int(X.shape[0] * 0.8)

X_train = X[:split]
y_train = y[:split]

X_test = X[split:]
y_test = y[split:]

In [6]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [8]:
# baggingclassifier > one base model > n times create
# voting classifier > different classifiers

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [10]:
clfs = [('LR', LogisticRegression()),
        ('KNN3', KNeighborsClassifier(n_neighbors=3)),
        ('KNN5', KNeighborsClassifier(n_neighbors=5)),
        ('KNN7', KNeighborsClassifier(n_neighbors=7)),
        ('KNN9', KNeighborsClassifier(n_neighbors=9)),
        ('SVM_l', SVC(kernel='linear')),
        ('SVM_r', SVC(kernel='rbf')),
        ('DT2', DecisionTreeClassifier(max_depth=2)),
        ('DT3', DecisionTreeClassifier(max_depth=3))]

In [11]:
# write code manually to see how each one of them acts seperately
for name, clf in clfs:
    clf.fit(X_train, y_train)
    print('{}:\ttrain_acc:{},\ttest_acc:{}'.format(
        name,
        clf.score(X_train, y_train),
        clf.score(X_test, y_test),
        ))

LR:	train_acc:0.871875,	test_acc:0.7375
KNN3:	train_acc:0.934375,	test_acc:0.8625
KNN5:	train_acc:0.928125,	test_acc:0.8875
KNN7:	train_acc:0.925,	test_acc:0.8875
KNN9:	train_acc:0.915625,	test_acc:0.8875
SVM_l:	train_acc:0.865625,	test_acc:0.725
SVM_r:	train_acc:0.909375,	test_acc:0.9
DT2:	train_acc:0.91875,	test_acc:0.9
DT3:	train_acc:0.91875,	test_acc:0.9


In [14]:
# clfs > list of dict
en_clf = VotingClassifier(clfs, n_jobs=-1)
# voting > hard,soft
# weights > the less the acc the less wi

# fit each of the models seperately
en_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('LR', LogisticRegression()),
                             ('KNN3', KNeighborsClassifier(n_neighbors=3)),
                             ('KNN5', KNeighborsClassifier()),
                             ('KNN7', KNeighborsClassifier(n_neighbors=7)),
                             ('KNN9', KNeighborsClassifier(n_neighbors=9)),
                             ('SVM_l', SVC(kernel='linear')), ('SVM_r', SVC()),
                             ('DT2', DecisionTreeClassifier(max_depth=2)),
                             ('DT3', DecisionTreeClassifier(max_depth=3))],
                 n_jobs=-1)

In [13]:
print(en_clf.score(X_train, y_train))
print(en_clf.score(X_test, y_test))

0.915625
0.8875
